In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
mu1 = np.array([-0.5, 1.2])
sigma1 = np.array([[1.0, 0.5], [0.5, 1.0]])
mu2 = np.array([1.0, -0.1])
sigma2 = np.array([[1.0, 0.2], [0.2, 1.0]])

states = np.array([0, 1, 1, 0, 1])[:, np.newaxis]
obs1 = np.random.multivariate_normal(mu1, sigma1, size=(states.size))
obs2 = np.random.multivariate_normal(mu2, sigma2, size=(states.size))

obs = obs1 * (1-states) + obs2 * states
obs

array([[-0.55711553,  1.07635327],
       [-0.13824083, -1.4178466 ],
       [ 1.46240073, -2.68252912],
       [-1.18024171,  0.89089846],
       [ 1.65176728, -0.79594064]])

In [3]:
B1 = tfd.MultivariateNormalFullCovariance(loc=mu1, covariance_matrix=sigma1) 
B2 = tfd.MultivariateNormalFullCovariance(loc=mu1, covariance_matrix=sigma2) 

2023-12-05 15:40:06.714962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 15:40:06.792015: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 15:40:06.809375: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 15:40:07.195268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


2023-12-05 15:40:07.903741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 15:40:07.921180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 15:40:07.921320: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 15:40:07.921816: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow wit

In [4]:
emissions = np.stack([B1.prob(obs), B2.prob(obs)], axis=-1)
emissions

array([[1.82374126e-01, 1.61111822e-01],
       [9.29017419e-04, 3.50957946e-03],
       [3.79295400e-09, 1.74009587e-06],
       [1.45723850e-01, 1.26890843e-01],
       [3.36390312e-05, 7.47622325e-04]])

In [5]:
emissions.shape

(5, 2)

In [6]:
A = np.array([[0.6, 0.4], [0.1, 0.9]])
P0 = np.array([0.5, 0.5])

In [7]:
# forward variables
alpha = np.zeros((states.size, 2))
alpha[0] = P0 * emissions[0]
for i in range(1, states.size):
    alpha[i] = np.matmul(alpha[i-1], A) * emissions[i]

# backward variables
beta = np.zeros((states.size, 2))
beta[-1] = 1
for i in range(states.size-2, -1, -1):
    beta[i] = np.matmul(A, beta[i+1] * emissions[i+1])

# likelihood
likelihood = np.sum(alpha[-1])

# state posterior probabilities
posterior = alpha * beta / likelihood

print("alpha=\n", alpha)
print("beta=\n", beta)
print("likelihood=\n", likelihood)
print("posterior=\n", posterior)

alpha=
 [[9.11870632e-02 8.05559109e-02]
 [5.83124065e-05 3.82456930e-04]
 [2.77769919e-13 6.39548223e-10]
 [9.34402956e-12 7.30516303e-11]
 [4.34333069e-16 5.19478488e-14]]
beta=
 [[2.11889453e-13 4.10405698e-13]
 [5.71318219e-11 1.28251533e-10]
 [6.22345133e-05 8.18779461e-05]
 [3.19232349e-04 6.76223996e-04]
 [1.00000000e+00 1.00000000e+00]]
likelihood=
 5.238218183123123e-14
posterior=
 [[3.68857812e-01 6.31142188e-01]
 [6.35997567e-02 9.36400243e-01]
 [3.30014427e-04 9.99669986e-01]
 [5.69452512e-02 9.43054749e-01]
 [8.29161851e-03 9.91708381e-01]]


### Test real embeddings

In [8]:
import sys
sys.path.insert(0, "..")
from learnMSA.protein_language_models import Common
from learnMSA.protein_language_models.BilinearSymmetric import make_scoring_model
from learnMSA.msa_hmm.SequenceDataset import SequenceDataset

In [9]:
config = Common.ScoringModelConfig(dim=32, lm_name="esm2", activation="sigmoid", scaled=False)
scoring_model = make_scoring_model(config, dropout=0.0, trainable=False)
scoring_model_path = Common.get_scoring_model_path(config)
scoring_model.load_weights(f"../learnMSA/protein_language_models/"+scoring_model_path)
bilinear_symmetric_layer = scoring_model.layers[-1]

In [10]:
language_model, encoder = Common.get_language_model(config.lm_name, max_len = 100, trainable=False)

/home/beckerf/mambaforge/envs/learnMSAdev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-05 15:40:11.494304: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [11]:
data = SequenceDataset("data/egf.fasta")
seq_batch = [data.get_standardized_seq(j) for j in range(10)]     
lm_inputs = encoder(seq_batch, np.repeat([[False, False]], len(seq_batch), axis=0))
emb = language_model(lm_inputs)
reduced_emb = bilinear_symmetric_layer._reduce(emb, training=False)
reduced_emb.shape

TensorShape([10, 34, 32])

In [12]:
seq_batch

['CDPNPCYNHGTCSLRATGYTCSCLPRYTGEH',
 'CDSQPCRNGTCSLNTNTDYTCACFPGYTGRY',
 'CDSDPCLNGGTCTLRSLDSYSCACAPGYTGQH',
 'CDHVTCLNGGTCQLKTLEDYTCACANGYTGER',
 'CDHVTCLNGGTCQLKTLEEYTCACANGYTGDR',
 'CDHVTCLNGGTCQLKTLEEYTCACANGYTGER',
 'CDHVTCLNGGTCQLKTLEEYTCACANGYAGER',
 'CDRHRCYNGGTCQLKTLQEATCACANGYTGEH',
 'CDRHRCYNGGTCQLKTLQEATCACANGYTGEH',
 'NACDRVRCQNGGTCQLKTLEDYTCSCANGYTGDH']

In [13]:
reduced_emb[:, 0]

<tf.Tensor: shape=(10, 32), dtype=float32, numpy=
array([[ 0.66692275,  0.04805628, -0.6304554 , -0.24984282,  0.05249589,
         0.15149611,  0.0278357 , -0.53607917,  0.5306717 ,  0.10943736,
         0.7228825 ,  0.58243716,  0.39630687, -0.47868562,  0.179137  ,
        -0.21374679,  0.02084704,  0.07543623, -0.37353945,  0.13151303,
         0.25534153, -0.0152976 , -0.25542912, -0.09959938,  0.304327  ,
         0.24813297, -1.4251356 ,  0.5276846 , -0.48343372, -0.41279402,
        -0.08422686,  0.5392273 ],
       [ 0.6235782 , -0.01863993, -0.6402881 , -0.16266158,  0.05353891,
         0.12203662,  0.07343104, -0.5461372 ,  0.5880978 ,  0.17223322,
         0.7533261 ,  0.5048562 ,  0.39819658, -0.5423788 ,  0.1654509 ,
        -0.22470437, -0.0192961 ,  0.01930928, -0.31966528,  0.1393848 ,
         0.1991722 , -0.04538697, -0.21309344, -0.09424427,  0.37025216,
         0.23474763, -1.4890687 ,  0.64384377, -0.41818753, -0.41294202,
        -0.1172558 ,  0.5249025 ],
    

In [14]:
from learnMSA.msa_hmm import Configuration, Align, Visualize, Emitter, Transitioner, Initializers, Training

Initializers.AminoAcidPlusMvnEmissionInitializer(config).global_emb

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
mu_large = Initializers.AminoAcidPlusMvnEmissionInitializer(config).global_emb
#mu_large = np.zeros(config.dim)
# sigma_large = np.random.normal(0, 0.001, (config.dim, config.dim))
# sigma_large = (sigma_large + sigma_large.T) / 2
# sigma_large = sigma_large * (1-np.eye(config.dim)) + np.eye(config.dim) * 1 # positive diagonal
sigma_large = np.eye(config.dim) * 1
mu_large = mu_large.astype(np.float32)
sigma_large = sigma_large.astype(np.float32)

In [16]:
B_large = tfd.MultivariateNormalFullCovariance(loc=mu_large, covariance_matrix=sigma_large) 

In [17]:
B_large.prob(reduced_emb[0]), B_large.log_prob(reduced_emb[0])

(<tf.Tensor: shape=(34,), dtype=float32, numpy=
 array([7.22866303e-15, 6.26343172e-15, 2.71328980e-14, 1.91586622e-14,
        1.34099232e-14, 1.27543663e-14, 2.84037015e-14, 1.88807134e-14,
        1.21130311e-14, 1.36527015e-14, 1.29331470e-14, 9.96147422e-15,
        2.20431583e-14, 4.08022174e-14, 1.76430559e-14, 3.55385379e-14,
        5.95116540e-14, 5.76013982e-14, 4.68330039e-14, 3.20261497e-14,
        3.21066992e-14, 3.60991517e-14, 4.97518354e-14, 4.83823525e-14,
        2.09175074e-14, 3.58900328e-14, 9.08748107e-15, 1.93054648e-14,
        1.69414856e-14, 1.48328479e-14, 1.08039247e-14, 1.69481508e-13,
        1.69481508e-13, 1.69481508e-13], dtype=float32)>,
 <tf.Tensor: shape=(34,), dtype=float32, numpy=
 array([-32.560722, -32.70405 , -31.23803 , -31.586021, -31.942781,
        -31.992903, -31.192257, -31.600636, -32.044495, -31.924839,
        -31.978983, -32.24005 , -31.445774, -30.83004 , -31.668434,
        -30.968159, -30.452604, -30.48523 , -30.692188, -31.072224

In [18]:
from learnMSA.msa_hmm import Emitter, MsaHmmCell
import tensorflow as tf

em = Emitter.MVNEmitter(config)
cell = MsaHmmCell.MsaHmmCell([20], emitter=[em])
em.cell_init(cell)
cell.build([1, reduced_emb.shape[-1]])
cell.recurrent_init()

Loading scoring model  new_scoring_models_frozen/esm2_32_sigmoid/checkpoints
Loading prior  /home/beckerf/brain/tmp_work/learnMSA/test/../learnMSA/msa_hmm/../protein_language_models/new_priors/esm2_32_reduced_mix100_sigmoid/checkpoints
Loading prior  /home/beckerf/brain/tmp_work/learnMSA/test/../learnMSA/msa_hmm/../protein_language_models/new_priors/esm2_32_reduced_mix100_sigmoid/checkpoints


In [30]:
em.mvn_log_pdf(tf.reshape(reduced_emb, (1, -1, 32)))[0, :34, 0]

<tf.Tensor: shape=(34,), dtype=float32, numpy=
array([-32.56023 , -32.703613, -31.238361, -31.58633 , -31.942822,
       -31.992249, -31.192743, -31.600811, -32.04517 , -31.924673,
       -31.978964, -32.23983 , -31.445961, -30.829882, -31.669186,
       -30.96814 , -30.45275 , -30.485249, -30.692434, -31.072147,
       -31.069647, -30.952482, -30.631992, -30.659657, -31.497744,
       -30.958572, -32.331703, -31.578377, -31.709175, -31.841656,
       -32.158894, -29.406033, -29.406033, -29.406033], dtype=float32)>

In [20]:
em.make_B()[0,:5,24:]

<tf.Tensor: shape=(5, 64), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [21]:
em.full_covariance

False

In [22]:
scale_kernel = em.B[:, :max(em.length)+1, len(SequenceDataset.alphabet)+em.scoring_model_config.dim:]
scale = tf.eye(em.scoring_model_config.dim, batch_shape=tf.shape(scale_kernel)[:-1]) * tf.math.softplus(scale_kernel + em.scale_diag_init)[..., tf.newaxis]
pinv = 1 / (scale+1e-6)

In [26]:
pinv

<tf.Tensor: shape=(1, 21, 32, 32), dtype=float32, numpy=
array([[[[9.9999905e-01, 1.0000000e+06, 1.0000000e+06, ...,
          1.0000000e+06, 1.0000000e+06, 1.0000000e+06],
         [1.0000000e+06, 9.9999905e-01, 1.0000000e+06, ...,
          1.0000000e+06, 1.0000000e+06, 1.0000000e+06],
         [1.0000000e+06, 1.0000000e+06, 9.9999905e-01, ...,
          1.0000000e+06, 1.0000000e+06, 1.0000000e+06],
         ...,
         [1.0000000e+06, 1.0000000e+06, 1.0000000e+06, ...,
          9.9999905e-01, 1.0000000e+06, 1.0000000e+06],
         [1.0000000e+06, 1.0000000e+06, 1.0000000e+06, ...,
          1.0000000e+06, 9.9999905e-01, 1.0000000e+06],
         [1.0000000e+06, 1.0000000e+06, 1.0000000e+06, ...,
          1.0000000e+06, 1.0000000e+06, 9.9999905e-01]],

        [[9.9999905e-01, 1.0000000e+06, 1.0000000e+06, ...,
          1.0000000e+06, 1.0000000e+06, 1.0000000e+06],
         [1.0000000e+06, 9.9999905e-01, 1.0000000e+06, ...,
          1.0000000e+06, 1.0000000e+06, 1.0000000e+06],

In [24]:
terminal = tf.cast(tf.reduce_all(reduced_emb == 0, axis=-1, keepdims=True), tf.float32)
fake_aa = tf.zeros_like(reduced_emb[...,:23])
full_input = tf.concat([fake_aa, terminal, reduced_emb, terminal], axis=-1)
full_input = full_input[tf.newaxis, ...]

In [25]:
em(full_input)[0]

<tf.Tensor: shape=(10, 34, 43), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 